In [1]:
import random
import math
import numpy as np
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from itertools import chain

In [2]:
class _LINE(object):

    def __init__(self, graph, rep_size=128, batch_size=1000, negative_ratio=5, order=3, cnt_u=15000, cnt_v=2529):
        self.cur_epoch = 0
        self.order = order
        self.g = graph
        self.node_size = graph.G.number_of_nodes()
        self.rep_size = rep_size
        self.batch_size = batch_size
        self.negative_ratio = negative_ratio

        self.gen_sampling_table()

    
        self.sess = tf.Session()
        cur_seed = random.getrandbits(32)
        initializer = tf.contrib.layers.xavier_initializer(
            uniform=False, seed=cur_seed)
        with tf.variable_scope("model", reuse=None, initializer=initializer):
            self.build_graph()
        self.sess.run(tf.global_variables_initializer())

    def build_graph(self):
        self.h = tf.placeholder(tf.int32, [None])
        self.t = tf.placeholder(tf.int32, [None])
        self.sign = tf.placeholder(tf.float32, [None])

        cur_seed = random.getrandbits(32)
        self.embeddings = tf.get_variable(name="embeddings"+str(self.order), shape=[
                                          self.node_size, self.rep_size], initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=cur_seed))
        self.context_embeddings = tf.get_variable(name="context_embeddings"+str(self.order), shape=[
                                                  self.node_size, self.rep_size], initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=cur_seed))

        self.h_e = tf.nn.embedding_lookup(self.embeddings, self.h)
        self.t_e = tf.nn.embedding_lookup(self.embeddings, self.t)
        self.t_e_context = tf.nn.embedding_lookup(
            self.context_embeddings, self.t)
        self.second_loss = -tf.reduce_mean(tf.log_sigmoid(
            self.sign*tf.reduce_sum(tf.multiply(self.h_e, self.t_e_context), axis=1)))
        self.first_loss = -tf.reduce_mean(tf.log_sigmoid(
            self.sign*tf.reduce_sum(tf.multiply(self.h_e, self.t_e), axis=1)))
        if self.order == 1:
            self.loss = self.first_loss
        else:
            self.loss = self.second_loss
        optimizer = tf.train.AdamOptimizer(0.001)
        self.train_op = optimizer.minimize(self.loss)

    def train_one_epoch(self):
        sum_loss = 0.0
        batches1 = self.batch_iter_heter()
        batches2 = self.batch_iter_homo(isM=True)
        batches3 = self.batch_iter_homo(isM=False)
        batch_id = 0
        for batch in chain(batches1, batches2, batches3):
            h, t, sign = batch
            feed_dict = {
                self.h: h,
                self.t: t,
                self.sign: sign,
            }
            _, cur_loss = self.sess.run([self.train_op, self.loss], feed_dict)
            sum_loss += cur_loss
            batch_id += 1
        print('epoch:{} sum of loss:{!s}'.format(self.cur_epoch, sum_loss))
        self.cur_epoch += 1

    def batch_iter_heter(self):
        look_up = self.g.look_up_dict

        table_size = 1e8
        numNodes = self.node_size

        edges = [(look_up[x[0]], look_up[x[1]]) for x in self.g.G.edges()]

        data_size = self.g.G.number_of_edges()
        edge_set = set([x[0]*numNodes+x[1] for x in edges])
        shuffle_indices = np.random.permutation(np.arange(data_size))

        # positive or negative mod
        mod = 0
        mod_size = 1 + self.negative_ratio
        h = []
        t = []
        sign = 0

        start_index = 0
        end_index = min(start_index+self.batch_size, data_size)
        while start_index < data_size:
            if mod == 0:
                sign = 1.
                h = []
                t = []
                for i in range(start_index, end_index):
                    if not random.random() < self.edge_prob[shuffle_indices[i]]:
                        shuffle_indices[i] = self.edge_alias[shuffle_indices[i]]
                    cur_h = edges[shuffle_indices[i]][0]
                    cur_t = edges[shuffle_indices[i]][1]
                    h.append(cur_h)
                    t.append(cur_t)
            else:
                sign = -1.
                t = []
                for i in range(len(h)):
                    if self.g.look_back_list[h[i]][0] == 'u':
                        t.append(self.item_table[random.randint(0, table_size-1)])
                    else:
                        t.append(self.user_table[random.randint(0, table_size-1)])

            yield h, t, [sign]
            mod += 1
            mod %= mod_size
            if mod == 0:
                start_index = end_index
                end_index = min(start_index+self.batch_size, data_size)
                
    def batch_iter_homo(self, isM = True):
        look_up = self.g.look_up_dict

        table_size = 1e8
        if isM:
            numNodes = self.g.M.number_of_nodes()
            edges = [(x[0], x[1]) for x in self.g.M.edges]
            data_size = self.g.M.number_of_edges()
            prob = self.prob_M
            alias = self.alias_M
        else:
            numNodes = self.g.N.number_of_nodes()
            bias = self.g.M.number_of_nodes()
            edges = [(x[0] + bias, x[1] + bias) for x in self.g.N.edges()]
            data_size = self.g.N.number_of_edges()
            prob = self.prob_N
            alias = self.alias_N
        

        shuffle_indices = np.random.permutation(np.arange(data_size))

        # positive or negative mod
        mod = 0
        mod_size = 1 + self.negative_ratio
        h = []
        t = []
        sign = 0

        start_index = 0
        end_index = min(start_index+self.batch_size, data_size)
        while start_index < data_size:
            if mod == 0:
                sign = 1.
                h = []
                t = []
                for i in range(start_index, end_index):
                    if not random.random() < prob[shuffle_indices[i]]:
                        shuffle_indices[i] = alias[shuffle_indices[i]]
                    cur_h = edges[shuffle_indices[i]][0]
                    cur_t = edges[shuffle_indices[i]][1]
                    h.append(cur_h)
                    t.append(cur_t)
            else:
                sign = -1.
                t = []
                for i in range(len(h)):
                    if isM:
                        t.append(self.uu_table[random.randint(0, table_size-1)])
                    else:
                        t.append(self.ii_table[random.randint(0, table_size-1)])

            yield h, t, [sign]
            mod += 1
            mod %= mod_size
            if mod == 0:
                start_index = end_index
                end_index = min(start_index+self.batch_size, data_size)

    def gen_sampling_table(self):
        table_size = 1e8
        power = 0.75
        numNodes = self.node_size

        print("Pre-procesing for non-uniform negative sampling!")
        pos_u = [i for i in range(numNodes) if self.g.look_back_list[i][0]=='u']
        pos_v = [i for i in range(numNodes) if self.g.look_back_list[i][0]=='i']
        degrees = np.array(self.g.G.degree())[:,1]
        degree_u = np.array(degrees[pos_u], dtype=np.int32)
        degree_v = np.array(degrees[pos_v], dtype=np.int32)
        
        self.user_table = self.get_distribution(degree_u)
        self.item_table = self.get_distribution(degree_v)
        
        degree_homo_u = np.array(self.g.M.degree())[:,1]
        degree_homo_v = np.array(self.g.N.degree())[:,1]
        
        self.uu_table = self.get_distribution(degree_homo_u)
        self.ii_table = self.get_distribution(degree_homo_v)

        
        self.edge_prob, self.edge_alias = self.alias_method(self.g.G)
        
        self.prob_M, self.alias_M = self.alias_method(self.g.M)
        
        self.prob_N, self.alias_N = self.alias_method(self.g.N)
        
    def get_distribution(self, degree_vec):
        table_size = 1e8
        power = 0.75
        
        x = np.power(degree_vec, power)
        norm = np.sum(x)
        print('The norm is', norm)
        
        table = np.zeros(int(table_size), dtype = np.uint32)
        
        
        p = 0
        i = 0
        num = len(degree_vec)
        for j in range(num):
            p = np.sum(x[0: j + 1])/norm
            # p += np.power(node_degree[j], power) / norm
            while i < table_size and (i / table_size) < p:
                table[i] = j
                i += 1
                
        return table
    
    def alias_method(self, G):
        data_size = G.number_of_edges()
        edge_alias = np.zeros(data_size, dtype=np.int32)
        edge_prob = np.zeros(data_size, dtype=np.float32)
        large_block = np.zeros(data_size, dtype=np.int32)
        small_block = np.zeros(data_size, dtype=np.int32)

        total_sum = sum([G[edge[0]][edge[1]]["weight"]
                         for edge in G.edges()])
        norm_prob = [G[edge[0]][edge[1]]["weight"] *
                     data_size/total_sum for edge in G.edges()]
        
        num_small_block = 0
        num_large_block = 0
        cur_small_block = 0
        cur_large_block = 0
        for k in range(data_size-1, -1, -1):
            if norm_prob[k] < 1:
                small_block[num_small_block] = k
                num_small_block += 1
            else:
                large_block[num_large_block] = k
                num_large_block += 1
        while num_small_block and num_large_block:
            num_small_block -= 1
            cur_small_block = small_block[num_small_block]
            num_large_block -= 1
            cur_large_block = large_block[num_large_block]
            
            edge_prob[cur_small_block] = norm_prob[cur_small_block]
            edge_alias[cur_small_block] = cur_large_block
            norm_prob[cur_large_block] = norm_prob[cur_large_block] + \
                norm_prob[cur_small_block] - 1
            if norm_prob[cur_large_block] < 1:
                small_block[num_small_block] = cur_large_block
                num_small_block += 1
            else:
                large_block[num_large_block] = cur_large_block
                num_large_block += 1

        while num_large_block:
            num_large_block -= 1
            edge_prob[large_block[num_large_block]] = 1
        while num_small_block:
            num_small_block -= 1
            edge_prob[small_block[num_small_block]] = 1
            
        return edge_prob, edge_alias

    def get_embeddings(self):
        vectors = {}
        embeddings = self.embeddings.eval(session=self.sess)
        # embeddings = self.sess.run(tf.nn.l2_normalize(self.embeddings.eval(session=self.sess), 1))
        look_back = self.g.look_back_list
        for i, embedding in enumerate(embeddings):
            vectors[look_back[i]] = embedding
        return vectors


class LINE(object):

    def __init__(self, graph, rep_size=128, batch_size=1000, epoch=10, negative_ratio=5, order=3, cnt_u=15000, cnt_v=2529, label_file=None, clf_ratio=0.5, auto_save=True):
        self.rep_size = rep_size
        self.order = order
        self.best_result = 0
        self.vectors = {}
        if order == 3:
            self.model1 = _LINE(graph, rep_size/2, batch_size,
                                negative_ratio, order=1)
            self.model2 = _LINE(graph, rep_size/2, batch_size,
                                negative_ratio, order=2)
            for i in range(epoch):
                self.model1.train_one_epoch()
                self.model2.train_one_epoch()
                if label_file:
                    self.get_embeddings()
                    X, Y = read_node_label(label_file)
                    print("Training classifier using {:.2f}% nodes...".format(
                        clf_ratio*100))
                    clf = Classifier(vectors=self.vectors,
                                     clf=LogisticRegression())
                    result = clf.split_train_evaluate(X, Y, clf_ratio)

                    if result['macro'] > self.best_result:
                        self.best_result = result['macro']
                        if auto_save:
                            self.best_vector = self.vectors

        else:
            self.model = _LINE(graph, rep_size, batch_size,
                               negative_ratio, order=self.order)
            for i in range(epoch):
                self.model.train_one_epoch()
                if label_file:
                    self.get_embeddings()
                    X, Y = read_node_label(label_file)
                    print("Training classifier using {:.2f}% nodes...".format(
                        clf_ratio*100))
                    clf = Classifier(vectors=self.vectors,
                                     clf=LogisticRegression())
                    result = clf.split_train_evaluate(X, Y, clf_ratio)

                    if result['macro'] > self.best_result:
                        self.best_result = result['macro']
                        if auto_save:
                            self.best_vector = self.vectors

        self.get_embeddings()
        if auto_save and label_file:
            self.vectors = self.best_vector

    def get_embeddings(self):
        self.last_vectors = self.vectors
        self.vectors = {}
        if self.order == 3:
            vectors1 = self.model1.get_embeddings()
            vectors2 = self.model2.get_embeddings()
            for node in vectors1.keys():
                self.vectors[node] = np.append(vectors1[node], vectors2[node])
        else:
            self.vectors = self.model.get_embeddings()

    def save_embeddings(self, filename):
        fout = open(filename, 'w')
        node_num = len(self.vectors.keys())
        fout.write("{} {}\n".format(node_num, self.rep_size))
        for node, vec in self.vectors.items():
            fout.write("{} {}\n".format(node,
                                        ' '.join([str(x) for x in vec])))
        fout.close()


In [3]:
from graphs import *

In [4]:
path = '../BiNE-master/data/wiki/rating_train_wiki.dat'

In [5]:
g = Graphs()
g.read_edgelist(path,weighted=True)
g.build_homo()

Constructing homo graphs....
The homo graph for users.....
The homo graph for items.....


In [6]:
g.G.number_of_nodes()

17529

In [7]:
g.M.number_of_edges()

43800

In [8]:
model = LINE(g, epoch = 100)

Pre-procesing for non-uniform negative sampling!
The norm is 44117.771519484086
The norm is 19349.29261155175
The norm is 27507.586321488227
The norm is 12222.21215052077
Pre-procesing for non-uniform negative sampling!
The norm is 44117.771519484086
The norm is 19349.29261155175
The norm is 27507.586321488227
The norm is 12222.21215052077
epoch:0 sum of loss:561.9734442532063
epoch:0 sum of loss:406.1935770511627
epoch:1 sum of loss:503.18235817551613
epoch:1 sum of loss:282.75578133109957
epoch:2 sum of loss:480.2685763128102
epoch:2 sum of loss:244.70883455313742
epoch:3 sum of loss:460.65317338658497
epoch:3 sum of loss:231.0335335917771
epoch:4 sum of loss:444.4130678907968
epoch:4 sum of loss:222.81088832602836
epoch:5 sum of loss:428.7160584559315
epoch:5 sum of loss:216.5587756393943
epoch:6 sum of loss:414.2424913667346
epoch:6 sum of loss:210.39481715776492
epoch:7 sum of loss:399.8439357805437
epoch:7 sum of loss:203.8012097242754
epoch:8 sum of loss:387.7898405694814
epoch:

In [9]:
model.save_embeddings('vec_md.dat')